In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr
import os

In [2]:
df_transactions = pd.read_csv(x + "transactions_ohne_vorname.csv")

### 1. Transactions Bereinigung

In [4]:
df_t_bereinigt = df_transactions[["iid", "key", "attrtype", "status", "pricetotal", "createdprovider"]]
df_t_bereinigt["pricetotal"] = pd.to_numeric(df_t_bereinigt["pricetotal"], errors = "coerce")
df_t_bereinigt = df_t_bereinigt.query('attrtype == "TI"')
df_t_bereinigt = df_t_bereinigt.query('status == "Buchung" or status == "Order"')
df_t_bereinigt = df_t_bereinigt.query('pricetotal >= 0.01')
searchfor = ['TEST']
#searchfor = ['TEST', 'JET', "WS"]
df_t_bereinigt = df_t_bereinigt[~df_t_bereinigt["key"].str.contains('|'.join(searchfor))]
df_t_bereinigt.head(20)

C:\Users\felix.oechslein\AppData\Local\Temp\ipykernel_7548\1557075601.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t_bereinigt["pricetotal"] = pd.to_numeric(df_t_bereinigt["pricetotal"], errors = "coerce")


,iid,key,attrtype,status,pricetotal,createdprovider
1,100279806,JET_1_a1179092,TI,Order,20.5,20220630111759
4,262428540,JET_1_a1179095,TI,Order,5.0,20220630113800
5,262429050,JET_1_a1179096,TI,Order,20.5,20220630115500
7,262429050,JET_1_a1179096,TI,Order,9.0,20220630115500
10,262429655,JET_1_a1179100,TI,Order,55.0,20220630121500
11,262429655,JET_1_a1179100,TI,Order,55.0,20220630121500
13,100520650,JET_1_a1179113,TI,Order,11.0,20220630123200
16,137606124,JET_1_a1179136,TI,Order,5.0,20220630130900
18,262430470,JET_1_a1179137,TI,Order,55.0,20220630131200
19,262430470,JET_1_a1179137,TI,Order,55.0,20220630131200


##### creation als datetime

In [5]:
df_t_bereinigt["createdprovider"] = pd.to_datetime(df_t_bereinigt['createdprovider'], format='%Y%m%d%H%M%S')
df_t_bereinigt.rename(columns = {"createdprovider" : "date"}, inplace = True)
df_t_bereinigt.head()

,iid,key,attrtype,status,pricetotal,date
1,100279806,JET_1_a1179092,TI,Order,20.5,2022-06-30 11:17:59
4,262428540,JET_1_a1179095,TI,Order,5.0,2022-06-30 11:38:00
5,262429050,JET_1_a1179096,TI,Order,20.5,2022-06-30 11:55:00
7,262429050,JET_1_a1179096,TI,Order,9.0,2022-06-30 11:55:00
10,262429655,JET_1_a1179100,TI,Order,55.0,2022-06-30 12:15:00


##### Einträge für jede iid im Zeitverlauf sortieren

In [6]:
df_t_bereinigt = df_t_bereinigt.sort_values(by=["iid", "date"])

##### Anzahl der unique Käufe 

In [8]:
df_t_bereinigt = df_t_bereinigt.loc[df_t_bereinigt.groupby(["iid", "key"])["date"].idxmin()]
df_t_bereinigt = df_t_bereinigt[["iid", "date"]]
df_t_bereinigt.head(20)

,iid,date
980434,100002747,2022-02-03 12:44:22
1896194,100002747,2022-04-29 14:03:00
835817,100009615,2018-09-10 09:52:00
835815,100009615,2018-10-22 10:32:00
896673,100009745,2021-10-29 11:52:50
896113,100009745,2021-10-29 11:55:06
941810,100009745,2021-12-16 15:36:50
1894067,100009745,2022-04-22 10:41:21
1908369,100009745,2022-05-16 15:41:14
1912092,100011333,2022-05-27 10:24:33


##### Monate als t = 1, .... abspeichern

In [9]:
# Berechnung der neuen Spalte "month_count"
df_t_bereinigt['month_count'] = (df_t_bereinigt['date'].dt.year - 2020) * 12 + df_t_bereinigt['date'].dt.month - 5
df_t_bereinigt.head()

,iid,date,month_count
980434,100002747,2022-02-03 12:44:22,21
1896194,100002747,2022-04-29 14:03:00,23
835817,100009615,2018-09-10 09:52:00,-20
835815,100009615,2018-10-22 10:32:00,-19
896673,100009745,2021-10-29 11:52:50,17


In [10]:
df_t_bereinigt[df_t_bereinigt['month_count'] >= 0]["month_count"].max()

31

#### 2. verarbeiteteDaten als Thalia_transactions_ersteBereinigung.csv in dem Ordner verarbeitete_Daten abspeichern

In [12]:
path = r'C:/Users/felix.oechslein/Desktop/Analysen Thalia, Mai/verarbeitete_daten/'
filename = 'Thalia_transactions_ersteBereinigung.csv'

df_t_bereinigt.to_csv(os.path.join(path, filename), index=False)